<a href="https://colab.research.google.com/github/rhiats/titanicPrediction/blob/main/titanicPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Titanic - Machine Learning from Disaster**